In [202]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [203]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import plotly.express as px
import seaborn as sns
from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [204]:
 # To ensure all the probabilistic things are reproducible
np.random.seed(42)

In [205]:
# Specify the path to the datasets
data_path = "./data/"

# Specify the filenames of the datasets
ratio_filename = 'ratio.csv'
federal_tax_filename = 'federal_tax.csv'
ca_tax_filename = 'ca_tax.csv'

# Read the CSV files and create backup copies
ratio_df_data = pd.read_csv(data_path + ratio_filename)
federal_tax_df_data = pd.read_csv(data_path + federal_tax_filename)
ca_tax_df_data = pd.read_csv(data_path + ca_tax_filename)

# Create working copies of the dataframes for analysis
ratio_df = ratio_df_data.copy()
federal_tax_df = federal_tax_df_data.copy()
ca_tax_df = ca_tax_df_data.copy()

In [206]:
pd.set_option('display.max_columns', None)
ratio_df.head()

,year,age,inflation_rate,saving_interest_rate,ira_interest_rate,ira_distribution_period
0,2023,62,0.08095,0.06,0.08,0.0
1,2024,63,0.04790,0.06,0.08,0.0
2,2025,64,0.03013,0.06,0.08,0.0
3,2026,65,0.02177,0.06,0.08,0.0
4,2027,66,0.02053,0.06,0.08,0.0


In [207]:
pd.set_option('display.max_columns', None)
federal_tax_df.head()

,bracket,over,up_to,base_tax,on_excess
0,1,0.0,22000.0,0.0,0.10
1,2,22000.0,89450.0,2200.0,0.12
2,3,89450.0,190750.0,10294.0,0.22
3,4,190750.0,364200.0,32580.0,0.24
4,5,364200.0,462500.0,74208.0,0.32


In [208]:
pd.set_option('display.max_columns', None)
ca_tax_df.head()

,bracket,over,up_to,base_tax,on_excess
0,1,0.0,20198.0,0.000,0.011
1,2,20198.0,47884.0,222.178,0.022
2,3,47884.0,75576.0,831.270,0.044
3,4,75576.0,104910.0,2049.718,0.066
4,5,104910.0,132590.0,3985.762,0.088


In [209]:
# Function to calculate tax based on tax brackets
def calculate_tax(income, tax_brackets):
    for bracket in tax_brackets:
        if income > bracket['over'] and income <= bracket['up_to']:
            return bracket['base_tax'] + ((income - bracket['over']) * bracket['on_excess'])
    return 0

In [210]:
# Implementing the financial projection with specific calculations and initial values for 2023

def financial_projection(ratio_df, ca_tax_df, federal_tax_df):

    # Initial values for 2023
    ma_balance_2023 = 580000
    ca_balance_2023 = 580000
    ira_balance_2023 = 400000
    annual_spending_2022 = 81500
    ma_tax_2023 = 0
    ca_tax_2023 = 0
    ma_federal_tax_2023 = 0
    ca_federal_tax_2023 = 0

    # Dictionary to store the results
    results = {}

    # Looping through each year
    for index, row in ratio_df.iterrows():
        year = row['year']
        age = row['age']

        # Setting the initial values for 2023
        if year == 2023:
            ma_balance = ma_balance_2023
            ca_balance = ca_balance_2023
            ira_balance = ira_balance_2023
            annual_spending = annual_spending_2022 * (1 + row['inflation_rate'])
            ma_tax = ma_tax_2023
            ca_tax = ca_tax_2023
            ma_federal_tax = ma_federal_tax_2023
            ca_federal_tax = ca_federal_tax_2023
        else:
            # Update values based on calculations for subsequent years
            annual_spending = results[year - 1]['Annual Spending'] * (1 + row['inflation_rate'])
            ma_balance = max(0, results[year - 1]['MA New Saving Balance'])
            ca_balance = max(0, results[year - 1]['CA New Saving Balance'])
            ira_balance = max(0, results[year - 1]['IRA Balance'] + results[year - 1]['IRA Interest'] - results[year - 1]['IRA Withdrawal'])
            ma_tax = results[year - 1]['MA Taxable Income'] * 0.05
            ca_tax = calculate_tax(results[year - 1]['CA Taxable Income'], ca_tax_df.to_dict('records'))
            ma_federal_tax = calculate_tax(results[year - 1]['MA Taxable Income'], federal_tax_df.to_dict('records'))
            ca_federal_tax = calculate_tax(results[year - 1]['CA Taxable Income'], federal_tax_df.to_dict('records'))

        # Calculate interests and IRA withdrawal
        ma_saving_interest = ma_balance * row['saving_interest_rate']
        ca_saving_interest = ca_balance * row['saving_interest_rate']
        ira_interest = ira_balance * row['ira_interest_rate']
        ira_withdrawal = 0 if year < 2034 else ira_balance / row['ira_distribution_period']

        # Calculate new balances and Tax delta
        ma_new_balance = ma_balance + ma_saving_interest + ira_withdrawal - annual_spending - ma_tax - ma_federal_tax
        ca_new_balance = ca_balance + ca_saving_interest + ira_withdrawal - annual_spending - ca_tax - ca_federal_tax
        ma_tax_delta_ca_tax = (ma_tax + ma_federal_tax) - (ca_tax + ca_federal_tax)

        # Store results for the current year
        results[year] = {
            'Year': year, 'Age': age, 'Inflation Rate': row['inflation_rate'], 'Annual Spending': annual_spending,
            'MA Saving Balance': ma_balance, 'CA Saving Balance': ca_balance, 'Saving Interest Rate': row['saving_interest_rate'],
            'MA Saving Interest': ma_saving_interest, 'CA Saving Interest': ca_saving_interest, 'IRA Balance': ira_balance,
            'IRA Interest Rate': row['ira_interest_rate'], 'IRA Interest': ira_interest, 'IRA Distribution Period': row['ira_distribution_period'],
            'IRA Withdrawal': ira_withdrawal, 'MA Taxable Income': ma_saving_interest + ira_withdrawal, 'MA Tax': ma_tax,
            'MA Federal Tax': ma_federal_tax, 'MA New Saving Balance': ma_new_balance, 'CA Taxable Income': ca_saving_interest + ira_withdrawal,
            'CA Tax': ca_tax, 'CA Federal Tax': ca_federal_tax, 'CA New Saving Balance': ca_new_balance, 'MA Tax Delta CA Tax': ma_tax_delta_ca_tax
        }

    return pd.DataFrame.from_dict(results, orient='index')

In [211]:
# Execute the financial_projection function to get the resulting DataFrame
financial_projection_df = financial_projection(ratio_df, ca_tax_df, federal_tax_df)
financial_projection_df

,Year,Age,Inflation Rate,Annual Spending,MA Saving Balance,CA Saving Balance,Saving Interest Rate,MA Saving Interest,CA Saving Interest,IRA Balance,IRA Interest Rate,IRA Interest,IRA Distribution Period,IRA Withdrawal,MA Taxable Income,MA Tax,MA Federal Tax,MA New Saving Balance,CA Taxable Income,CA Tax,CA Federal Tax,CA New Saving Balance,MA Tax Delta CA Tax
2023.0,2023.0,62.0,0.08095,88097.425000,580000.000000,580000.000000,0.06,34800.000000,34800.000000,4.000000e+05,0.08,32000.000000,0.0,0.000000,34800.000000,0.000000,0.000000,526702.575000,34800.000000,0.000000,0.000000,526702.575000,0.000000
2024.0,2024.0,63.0,0.04790,92317.291658,526702.575000,526702.575000,0.06,31602.154500,31602.154500,4.320000e+05,0.08,34560.000000,0.0,0.000000,31602.154500,1740.000000,3736.000000,460511.437842,31602.154500,543.422000,3736.000000,461708.015842,1196.578000
2025.0,2025.0,64.0,0.03013,95098.811655,460511.437842,461708.015842,0.06,27630.686271,27702.480951,4.665600e+05,0.08,37324.800000,0.0,0.000000,27630.686271,1580.107725,3352.258540,388110.946193,27702.480951,473.069399,3352.258540,390486.357199,1107.038326
2026.0,2026.0,65.0,0.02177,97169.112785,388110.946193,390486.357199,0.06,23286.656772,23429.181432,5.038848e+05,0.08,40310.784000,0.0,0.000000,23286.656772,1381.534314,2875.682352,309971.273514,23429.181432,387.276581,2884.297714,313474.851551,985.642371
2027.0,2027.0,66.0,0.02053,99163.994670,309971.273514,313474.851551,0.06,18598.276411,18808.491093,5.441956e+05,0.08,43535.646720,0.0,0.000000,18598.276411,1164.332839,2354.398813,225886.823603,18808.491093,293.263992,2371.501772,230454.582210,853.965888
2028.0,2028.0,67.0,0.02062,101208.756240,225886.823603,230454.582210,0.06,13553.209416,13827.274933,5.877312e+05,0.08,47018.498458,0.0,0.000000,13553.209416,929.913821,1859.827641,135441.535317,13827.274933,206.893402,1880.849109,140985.358391,701.998950
2029.0,2029.0,68.0,0.02185,103420.167564,135441.535317,140985.358391,0.06,8126.492119,8459.121503,6.347497e+05,0.08,50779.978334,0.0,0.000000,8126.492119,677.660471,1355.320942,38114.878459,8459.121503,152.100024,1382.727493,44489.484813,498.153895
2030.0,2030.0,69.0,0.02254,105751.258141,38114.878459,44489.484813,0.06,2286.892708,2669.369089,6.855297e+05,0.08,54842.376601,0.0,0.000000,2286.892708,406.324606,812.649212,-66568.460792,2669.369089,93.050337,845.912150,-59531.366726,280.011331
2031.0,2031.0,70.0,0.02269,108150.754188,0.000000,0.000000,0.06,0.000000,0.000000,7.403721e+05,0.08,59229.766729,0.0,0.000000,0.000000,114.344635,228.689271,-108493.788095,0.000000,29.363060,266.936909,-108447.054157,46.733937
2032.0,2032.0,71.0,0.02273,110609.020831,0.000000,0.000000,0.06,0.000000,0.000000,7.996019e+05,0.08,63968.148067,0.0,0.000000,0.000000,0.000000,0.000000,-110609.020831,0.000000,0.000000,0.000000,-110609.020831,0.000000


# Assumption
In making calculations and proposals, the following assumptions were made.

    -Did not take into account the 2022 tax that needs to be paid in 2023
    -Annual spending is assumed to be paid at the end of the year
    -Interest rates on both the Saving account and IRA account are assumed to remain unchanged
    -Special expenses like air, shipping, etc. associated with moving are not included in the calculations
    -No income other than interest income from the Saving Account and IRA Account is assumed
    -IRA withdrawals are assumed to be made in a lump sum at the end of the year
    -More than one move (e.g., to CA and back to MA) is not taken into account
    -No withdrawals from IRA accounts prior to age 73
    -Life expectancy is assumed to be 82 based on retiree emails (78.5 + 3)
    -'Appendix B. Uniform Lifetime Table' of the following website was referenced for the IRA distribution period
    https://www.irs.gov/publications/p590b#en_US_2022_publink100090310

# Q1: Do they have enough money to last to their estimated life expectation?

In [212]:
# Filter rows where MA New Saving Balance is less than or equal to 0
negative_balance_years = financial_projection_df[financial_projection_df['MA New Saving Balance'] <= 0]

# Get the list of years with negative balances
negative_balance_years_list = negative_balance_years['Year'].tolist()
print("Years when MA New Saving Balance falls below zero:", negative_balance_years_list)

Years when MA New Saving Balance falls below zero: [2030.0, 2031.0, 2032.0, 2033.0, 2034.0, 2035.0, 2036.0, 2037.0, 2038.0, 2039.0, 2040.0, 2041.0, 2042.0, 2043.0]


In [213]:
# Filter rows where MA New Saving Balance is less than or equal to 0
negative_balance_years = financial_projection_df[financial_projection_df['CA New Saving Balance'] <= 0]

# Get the list of years with negative balances
negative_balance_years_list = negative_balance_years['Year'].tolist()
print("Years when CA New Saving Balance falls below zero:", negative_balance_years_list)

Years when CA New Saving Balance falls below zero: [2030.0, 2031.0, 2032.0, 2033.0, 2034.0, 2035.0, 2036.0, 2037.0, 2038.0, 2039.0, 2040.0, 2041.0, 2042.0, 2043.0]


# A1:
Based on this retiree's current assumed annual spending and planned withdrawals from his IRA account, he will have a negative savings balance in 2030, when he turns 69, even if he continues to live in Massachusetts or moves to California beginning in 2023.

# Q2: If not, should they adjust their spending? by how much?

In [214]:
# Updating the financial projection with specific calculations

def financial_projection2(ratio_df, ca_tax_df, federal_tax_df, annual_spending_2022):
    # Initialize variables for 2023 and other years
    ma_balance_2023 = 580000
    ca_balance_2023 = 580000
    ira_balance_2023 = 400000
    ma_tax_2023 = 0
    ca_tax_2023 = 0
    ma_federal_tax_2023 = 0
    ca_federal_tax_2023 = 0
    results = {}

    # Loop through each year
    for index, row in ratio_df.iterrows():
        year = row['year']
        age = row['age']

        # Set initial values for 2023 or update for subsequent years
        if year == 2023:
            annual_spending = annual_spending_2022 * (1 + row['inflation_rate'])
            ma_balance = ma_balance_2023
            ca_balance = ca_balance_2023
            ira_balance = ira_balance_2023
            ma_tax = ma_tax_2023
            ca_tax = ca_tax_2023
            ma_federal_tax = ma_federal_tax_2023
            ca_federal_tax = ca_federal_tax_2023
        else:
            annual_spending *= (1 + row['inflation_rate'])
            # Update balances and taxes based on previous year's results
            ma_balance = max(0, results[year - 1]['MA New Saving Balance'])
            ca_balance = max(0, results[year - 1]['CA New Saving Balance'])
            ira_balance = max(0, results[year - 1]['IRA Balance'] + results[year - 1]['IRA Interest'] - results[year - 1]['IRA Withdrawal'])
            ma_tax = results[year - 1]['MA Taxable Income'] * 0.05
            ca_tax = calculate_tax(results[year - 1]['CA Taxable Income'], ca_tax_df.to_dict('records'))
            ma_federal_tax = calculate_tax(results[year - 1]['MA Taxable Income'], federal_tax_df.to_dict('records'))
            ca_federal_tax = calculate_tax(results[year - 1]['CA Taxable Income'], federal_tax_df.to_dict('records'))

        # Calculate interests and IRA withdrawal
        ma_saving_interest = ma_balance * row['saving_interest_rate']
        ca_saving_interest = ca_balance * row['saving_interest_rate']
        ira_interest = ira_balance * row['ira_interest_rate']
        ira_withdrawal = 0 if year < 2034 else ira_balance / row['ira_distribution_period']

        # Calculate new balances and Tax delta
        ma_new_balance = ma_balance + ma_saving_interest + ira_withdrawal - annual_spending - ma_tax - ma_federal_tax
        ca_new_balance = ca_balance + ca_saving_interest + ira_withdrawal - annual_spending - ca_tax - ca_federal_tax
        ma_tax_delta_ca_tax = (ma_tax + ma_federal_tax) - (ca_tax + ca_federal_tax)

        # Store results for the current year
        results[year] = {
            'Year': year, 'Age': age, 'Inflation Rate': row['inflation_rate'], 'Annual Spending': annual_spending,
            'MA Saving Balance': ma_balance, 'CA Saving Balance': ca_balance, 'Saving Interest Rate': row['saving_interest_rate'],
            'MA Saving Interest': ma_saving_interest, 'CA Saving Interest': ca_saving_interest, 'IRA Balance': ira_balance,
            'IRA Interest Rate': row['ira_interest_rate'], 'IRA Interest': ira_interest, 'IRA Distribution Period': row['ira_distribution_period'],
            'IRA Withdrawal': ira_withdrawal, 'MA Taxable Income': ma_saving_interest + ira_withdrawal, 'MA Tax': ma_tax,
            'MA Federal Tax': ma_federal_tax, 'MA New Saving Balance': ma_new_balance, 'CA Taxable Income': ca_saving_interest + ira_withdrawal,
            'CA Tax': ca_tax, 'CA Federal Tax': ca_federal_tax, 'CA New Saving Balance': ca_new_balance, 'MA Tax Delta CA Tax': ma_tax_delta_ca_tax
        }

    return pd.DataFrame.from_dict(results, orient='index')

In [215]:
# Function to find the maximum annual spending for 2022
def find_max_annual_spending_2022(ratio_df, ca_tax_df, federal_tax_df):
    max_spending = 100000  # Start with an initial guess
    decrement = 500       # Decrement to adjust spending

    while max_spending > 0:
        projection_df = financial_projection2(ratio_df, ca_tax_df, federal_tax_df, max_spending)
        balance_2033 = projection_df.at[2033, 'MA New Saving Balance']

        if balance_2033 >= 0:
            return max_spending
        max_spending -= decrement

    return 0

max_annual_spending_2022 = find_max_annual_spending_2022(ratio_df, ca_tax_df, federal_tax_df)
print("Maximum annual spending in 2022 to keep MA New Saving Balance above zero in 2033:", max_annual_spending_2022)

Maximum annual spending in 2022 to keep MA New Saving Balance above zero in 2033: 57000


In [216]:
# Function to find the maximum annual spending for 2022
def find_max_annual_spending_2022(ratio_df, ca_tax_df, federal_tax_df):
    max_spending = 100000  # Start with an initial guess
    decrement = 500       # Decrement to adjust spending

    while max_spending > 0:
        projection_df = financial_projection2(ratio_df, ca_tax_df, federal_tax_df, max_spending)
        balance_2033 = projection_df.at[2033, 'CA New Saving Balance']

        if balance_2033 >= 0:
            return max_spending
        max_spending -= decrement

    return 0

max_annual_spending_2022 = find_max_annual_spending_2022(ratio_df, ca_tax_df, federal_tax_df)
print("Maximum annual spending in 2022 to keep CA New Saving Balance above zero in 2033:", max_annual_spending_2022)

Maximum annual spending in 2022 to keep CA New Saving Balance above zero in 2033: 57500


# A2:
If this retiree has no other source of income and does not accelerate the age at which withdrawals from the IRA account begin, then a reduction in annual spending is mandatory. Since withdrawals from IRA accounts will begin in 2034, the goal is to have a saving balance that is not negative by 2033. Based on the above calculations, this retiree would need to reduce his annual assumed spendings as of January 1 to USD 57,000 if he is in Massachusetts beginning in 2023, and to USD 575,000 even if he moves to California.

# Q3: Any other recommendation outside of their questions that could benefit them?

In [217]:
def financial_projection3(ratio_df, ca_tax_df, federal_tax_df, decrease_amount):
    # Initialize variables for 2023 and other years
    ma_balance_2023 = 580000
    ca_balance_2023 = 580000
    ira_balance_2023 = 400000
    annual_spending_2022 = 57000
    ma_tax_2023 = 0
    ca_tax_2023 = 0
    ma_federal_tax_2023 = 0
    ca_federal_tax_2023 = 0
    results = {}

    # Loop through each year
    for index, row in ratio_df.iterrows():
        year = row['year']
        age = row['age']

        # Set initial values for 2023 or update for subsequent years
        if year == 2023:
            annual_spending = annual_spending_2022 * (1 + row['inflation_rate'])
            ma_balance = ma_balance_2023
            ca_balance = ca_balance_2023
            ira_balance = ira_balance_2023
            ma_tax = ma_tax_2023
            ca_tax = ca_tax_2023
            ma_federal_tax = ma_federal_tax_2023
            ca_federal_tax = ca_federal_tax_2023
        else:
            annual_spending *= (1 + row['inflation_rate'])
            # Update balances and taxes based on previous year's results
            ma_balance = max(0, results[year - 1]['MA New Saving Balance'])
            ca_balance = max(0, results[year - 1]['CA New Saving Balance'])
            ira_balance = max(0, results[year - 1]['IRA Balance'] + results[year - 1]['IRA Interest'] - results[year - 1]['IRA Withdrawal'])
            ma_tax = results[year - 1]['MA Taxable Income'] * 0.05
            ca_tax = calculate_tax(results[year - 1]['CA Taxable Income'], ca_tax_df.to_dict('records'))
            ma_federal_tax = calculate_tax(results[year - 1]['MA Taxable Income'], federal_tax_df.to_dict('records'))
            ca_federal_tax = calculate_tax(results[year - 1]['CA Taxable Income'], federal_tax_df.to_dict('records'))

        # Apply the adjusted IRA Distribution Period from 2034 to 2043 and Calculate IRA withdrawal based on the distribution period
        if 2034 <= year <= 2043:
            ira_distribution_period = row['ira_distribution_period'] - decrease_amount
            ira_withdrawal = ira_balance / ira_distribution_period
        else:
            ira_distribution_period = row['ira_distribution_period']
            ira_withdrawal = 0
        
        # Calculate interests and IRA withdrawal
        ma_saving_interest = ma_balance * row['saving_interest_rate']
        ca_saving_interest = ca_balance * row['saving_interest_rate']
        ira_interest = ira_balance * row['ira_interest_rate']

        # Calculate new balances and Tax delta
        ma_new_balance = ma_balance + ma_saving_interest + ira_withdrawal - annual_spending - ma_tax - ma_federal_tax
        ca_new_balance = ca_balance + ca_saving_interest + ira_withdrawal - annual_spending - ca_tax - ca_federal_tax
        ma_tax_delta_ca_tax = (ma_tax + ma_federal_tax) - (ca_tax + ca_federal_tax)

        # Store results for the current year
        results[year] = {
            'Year': year, 'Age': age, 'Inflation Rate': row['inflation_rate'], 'Annual Spending': annual_spending,
            'MA Saving Balance': ma_balance, 'CA Saving Balance': ca_balance, 'Saving Interest Rate': row['saving_interest_rate'],
            'MA Saving Interest': ma_saving_interest, 'CA Saving Interest': ca_saving_interest, 'IRA Balance': ira_balance,
            'IRA Interest Rate': row['ira_interest_rate'], 'IRA Interest': ira_interest, 'IRA Distribution Period': row['ira_distribution_period'],
            'IRA Withdrawal': ira_withdrawal, 'MA Taxable Income': ma_saving_interest + ira_withdrawal, 'MA Tax': ma_tax,
            'MA Federal Tax': ma_federal_tax, 'MA New Saving Balance': ma_new_balance, 'CA Taxable Income': ca_saving_interest + ira_withdrawal,
            'CA Tax': ca_tax, 'CA Federal Tax': ca_federal_tax, 'CA New Saving Balance': ca_new_balance, 'MA Tax Delta CA Tax': ma_tax_delta_ca_tax
        }

    return pd.DataFrame.from_dict(results, orient='index')

def find_min_decrease_for_positive_balance(ratio_df, ca_tax_df, federal_tax_df):
    for decrease_amount in np.arange(0.5, 18, 0.5):
        projection_df = financial_projection3(ratio_df, ca_tax_df, federal_tax_df, decrease_amount)
        # Check if MA New Saving Balance is positive for each year from 2034 to 2043
        if all(projection_df.loc[2034:2043, 'MA New Saving Balance'] >= 0):
            return decrease_amount
    return None  # Return None if no suitable decrease amount is found

# Find the minimum decrease amount
min_decrease = find_min_decrease_for_positive_balance(ratio_df, ca_tax_df, federal_tax_df)
print("Minimum decrease in IRA Distribution Period for a positive MA New Saving Balance from 2034 to 2043:", min_decrease)

Minimum decrease in IRA Distribution Period for a positive MA New Saving Balance from 2034 to 2043: 15.5


In [218]:
def financial_projection3(ratio_df, ca_tax_df, federal_tax_df, decrease_amount):
    # Initialize variables for 2023 and other years
    ma_balance_2023 = 580000
    ca_balance_2023 = 580000
    ira_balance_2023 = 400000
    annual_spending_2022 = 57000
    ma_tax_2023 = 0
    ca_tax_2023 = 0
    ma_federal_tax_2023 = 0
    ca_federal_tax_2023 = 0
    results = {}

    # Loop through each year
    for index, row in ratio_df.iterrows():
        year = row['year']
        age = row['age']

        # Set initial values for 2023 or update for subsequent years
        if year == 2023:
            annual_spending = annual_spending_2022 * (1 + row['inflation_rate'])
            ma_balance = ma_balance_2023
            ca_balance = ca_balance_2023
            ira_balance = ira_balance_2023
            ma_tax = ma_tax_2023
            ca_tax = ca_tax_2023
            ma_federal_tax = ma_federal_tax_2023
            ca_federal_tax = ca_federal_tax_2023
        else:
            annual_spending *= (1 + row['inflation_rate'])
            # Update balances and taxes based on previous year's results
            ma_balance = max(0, results[year - 1]['MA New Saving Balance'])
            ca_balance = max(0, results[year - 1]['CA New Saving Balance'])
            ira_balance = max(0, results[year - 1]['IRA Balance'] + results[year - 1]['IRA Interest'] - results[year - 1]['IRA Withdrawal'])
            ma_tax = results[year - 1]['MA Taxable Income'] * 0.05
            ca_tax = calculate_tax(results[year - 1]['CA Taxable Income'], ca_tax_df.to_dict('records'))
            ma_federal_tax = calculate_tax(results[year - 1]['MA Taxable Income'], federal_tax_df.to_dict('records'))
            ca_federal_tax = calculate_tax(results[year - 1]['CA Taxable Income'], federal_tax_df.to_dict('records'))

        # Apply the adjusted IRA Distribution Period from 2034 to 2043 and Calculate IRA withdrawal based on the distribution period
        if 2034 <= year <= 2043:
            ira_distribution_period = row['ira_distribution_period'] - decrease_amount
            ira_withdrawal = ira_balance / ira_distribution_period
        else:
            ira_distribution_period = row['ira_distribution_period']
            ira_withdrawal = 0
        
        # Calculate interests and IRA withdrawal
        ma_saving_interest = ma_balance * row['saving_interest_rate']
        ca_saving_interest = ca_balance * row['saving_interest_rate']
        ira_interest = ira_balance * row['ira_interest_rate']

        # Calculate new balances and Tax delta
        ma_new_balance = ma_balance + ma_saving_interest + ira_withdrawal - annual_spending - ma_tax - ma_federal_tax
        ca_new_balance = ca_balance + ca_saving_interest + ira_withdrawal - annual_spending - ca_tax - ca_federal_tax
        ma_tax_delta_ca_tax = (ma_tax + ma_federal_tax) - (ca_tax + ca_federal_tax)

        # Store results for the current year
        results[year] = {
            'Year': year, 'Age': age, 'Inflation Rate': row['inflation_rate'], 'Annual Spending': annual_spending,
            'MA Saving Balance': ma_balance, 'CA Saving Balance': ca_balance, 'Saving Interest Rate': row['saving_interest_rate'],
            'MA Saving Interest': ma_saving_interest, 'CA Saving Interest': ca_saving_interest, 'IRA Balance': ira_balance,
            'IRA Interest Rate': row['ira_interest_rate'], 'IRA Interest': ira_interest, 'IRA Distribution Period': row['ira_distribution_period'],
            'IRA Withdrawal': ira_withdrawal, 'MA Taxable Income': ma_saving_interest + ira_withdrawal, 'MA Tax': ma_tax,
            'MA Federal Tax': ma_federal_tax, 'MA New Saving Balance': ma_new_balance, 'CA Taxable Income': ca_saving_interest + ira_withdrawal,
            'CA Tax': ca_tax, 'CA Federal Tax': ca_federal_tax, 'CA New Saving Balance': ca_new_balance, 'MA Tax Delta CA Tax': ma_tax_delta_ca_tax
        }

    return pd.DataFrame.from_dict(results, orient='index')

def find_min_decrease_for_positive_balance(ratio_df, ca_tax_df, federal_tax_df):
    for decrease_amount in np.arange(0.5, 18, 0.5):
        projection_df = financial_projection3(ratio_df, ca_tax_df, federal_tax_df, decrease_amount)
        # Check if MA New Saving Balance is positive for each year from 2034 to 2043
        if all(projection_df.loc[2034:2043, 'CA New Saving Balance'] >= 0):
            return decrease_amount
    return None  # Return None if no suitable decrease amount is found

# Find the minimum decrease amount
min_decrease = find_min_decrease_for_positive_balance(ratio_df, ca_tax_df, federal_tax_df)
print("Minimum decrease in IRA Distribution Period for a positive CA New Saving Balance from 2034 to 2043:", min_decrease)

Minimum decrease in IRA Distribution Period for a positive CA New Saving Balance from 2034 to 2043: 15.0


# A3:
Even if the assumed annual spendings as of January 1, 2023 were lowered as per the answer in Q2, the saving balance would still be negative in 2034. Also, there would be too much money in the IRA account in 2043, when this retiree reaches the age of 82, the assumed life expectancy of this retiree. However, we do not recommend that the IRA account be paid out earlier than 2034, as this would reduce the interest on the account.

The proposal is to withdraw more money from the IRA account by decreasing the value of IRA distribution period after 2034. In the prompt above, we calculated how much smaller the IRA distribution period from 2034 to 2043 would be to keep a positive saving balance from 2034 to 2043.

To simplify the calculation, the amount of IRA distribution period decrease was set uniformly from 2034 to 2043. If the IRA distribution period were to be drawn down more than necessary, interest income would be reduced by that amount, and also, there would still be a risk of financial loss if this retiree lived longer than the assumed life expectancy. Therefore, the minimum amount to be reduced was considered in 0.5 increments in order to maintain a positive savings balance from 2034 to 2043.

The results of the above calculations indicate that the ira distribution period should be smaller after 2034, 15.5 if this retiree continues to live in MA and 15.0 if he moves to CA to live. This would keep the saving balance positive until 2043, when this retiree would reach the assumed life expectancy of 82 years.

# Q4: Move to CA or stay in MA?

In [219]:
# Uniformly decrease the IRA distribution period from 2034 to 2043 by 15.5
decrease_amount = 15.5

# Calculate financial projections using the financial_projection3 function
projection_df = financial_projection3(ratio_df, ca_tax_df, federal_tax_df, decrease_amount)

# Add a column for the running total of MA Tax Delta CA Tax
projection_df['MA Tax Delta CA Tax Running Total'] = projection_df['MA Tax Delta CA Tax'].cumsum()

# Extract and display only the required columns including the running total
selected_columns = ['Year', 'Age', 'MA New Saving Balance', 'CA New Saving Balance', 'MA Tax Delta CA Tax', 'MA Tax Delta CA Tax Running Total']
output_df = projection_df.loc[2034:2043, selected_columns]
print(output_df)

          Year   Age  MA New Saving Balance  CA New Saving Balance  \
2034.0  2034.0  73.0            7684.557295           19965.588726   
2035.0  2035.0  74.0            3588.740057           18056.157973   
2036.0  2036.0  75.0            2810.077633           19389.749071   
2037.0  2037.0  76.0            5795.524003           24467.061622   
2038.0  2038.0  77.0           11516.341731           32177.497859   
2039.0  2039.0  78.0           22308.378171           44782.623285   
2040.0  2040.0  79.0           38409.662599           62429.385776   
2041.0  2041.0  80.0           60417.131938           85648.915821   
2042.0  2042.0  81.0           85040.858199          110968.573778   
2043.0  2043.0  82.0          117249.643944          143489.714466   

        MA Tax Delta CA Tax  MA Tax Delta CA Tax Running Total  
2034.0           116.816392                        8353.883389  
2035.0          1449.524599                        9803.407988  
2036.0          1244.208447       

# A4:
We would like to propose to this retiree that he not move to CA and continue to live in MA for all of 2023 and beyond. The above calculations show how much of a difference in tax would be created between MA and CA when annual spending is lowered to USD 57,000 and the IRA distribution period after 2034 is lowered uniformly by 15.5.

The calculation results show that the MA tax is higher until 2040, and the CA tax is higher only for the three years beginning in 2041. And the running total of the difference between MA tax and CA tax shows that in the 21 years from 2023 to 2043, this retiree would have to pay USD 11,494 more in taxes to continue living in MA than to move to CA.

This value alone suggests that moving to CA starting in 2023 would result in a tax saving. However, considering that special expenses like air, shipping, etc. associated with the move are not included in the calculation, it is not likely to make a significant difference from a financial standpoint. On top of that, given the age of this retiree and the fact that his grandchildren are in MA, we would suggest that he stay in MA instead of moving to CA.

In addition, the above calculation lowers the IRA distribution period after 2034 by a uniform 15.5, but the closer this value is to 18.5 and the more money is deducted from the IRA account until 2043, the more tax savings will be realized by staying in MA rather than CA.

# Q5: How much money would they save on taxes a result of your recommendation?

# A5:
First, in considering the retirement plan this retiree, changes to the annual spending and IRA distribution period answered in Q2 and Q3 are necessary to have a positive saving balance by 2043, the assumed life span.

In addition, as we answered in Q4, we are proposing to continue to live in MA where the tax rate is high. Therefore, it is not appropriate to answer how much tax savings can be made under this proposal. However, this proposal will not only keep the saving balance positive until 2043, but also, considering the financial situation including moving expenses, it is preferable to continue to live in MA instead of moving to CA.

# Conclustion
In conclusion, our recommendations for this retiree are as follows.

    -Make an effort to reduce the assumed annual spending as of January 1, 2023 from USD 81,500 to USD 57,000
    -Continue to live in MA instead of moving to CA
    -Not make withdrawals from their IRA account prior to age 73
    -Withdraw more money by reducing the value of the IRA distribution period after 2034 by at least 15.5 than specified
    -Consider ways to create a new source of income by selling or renting out your CA home to make financial planning more affordable